In [8]:
# import python packages
import pandas as pd
import numpy as np

# import src code
import ase_evaluation as ae
#import benchmark_posteriors as bp
import bootstrap as btstrp
import cross_validation as cv
import logistic_regression as lr
import naive_bayes as nb
import network as ntwk
import process as prcs
#import RIVER as rvr
import simulate_data as sim
import benchmark_posteriors as bnchmk

In [9]:
# generate simulated data
s = sim.SimulateData("./test_output/", 'with_transfer', 0.4, 0.6, 0.01)
s._run()

In [10]:
# create a process object
p = prcs.Process('./test_output', 0.1)
#p = prcs.Process('../simulated_data/tied_stronger_effects/data/', 0.1)

# process simulated data
p._process_simulated_data()

In [ ]:
p.train_list[0].head()

In [ ]:
# create a bootstrap object
b = btstrp.Bootstrap(p.train_list, p.tissues, p.genomic_features, num_simulations=2, num_folds=3)
lambda_hp_children_dict, lambda_hp_parent = b._run_bootstrap()

In [ ]:
"tissue-specific transfer factors: ", lambda_hp_children_dict, "global transfer factor: ", lambda_hp_parent

In [11]:
n = ntwk.Network(p.train_list, p.test_list, p.tissues, p.genomic_features, 
                 with_transfer=True, output_dir="SPEER_output", 
                 lambda_hp_parent = None,
                 lambda_hp_children_dict = None,
                 e_distribution = 'cat')
train_list, test_list, beta_parent, beta_children, phi = n.run()

In [ ]:
train_list[0].head()

In [12]:
lambda_hp_children_dict = {'brain': 0.01, 'group1': 0.01, 'muscle': 0.01, 'epithelial': 0.01, 'digestive': 0.01}
n = ntwk.Network(train_list, test_list, p.tissues, p.genomic_features, 
                 with_transfer=False, output_dir="SPEER_output", 
                 lambda_hp_parent = None, 
                 lambda_hp_children_dict = lambda_hp_children_dict, 
                 e_distribution = 'cat')
train_list, test_list, beta_parent, beta_children, phi = n.run()

In [18]:
bn = bnchmk.BenchmarkPosteriors(train_list, test_list, p.genomic_features)
train_list, test_list = bn.fit_models()

TypeError: 'NoneType' object is not iterable

In [14]:
train_list[0].head()

,0,1,2,3,4,5,6,7,8,9,...,20,expr_label,median_expr_label,z_label,median_z_label,SPEER,SPEER without transfer,shared tissue genome only,tissue specific genome only,tissue
8159,1.0,0.398283,0.496448,0.223423,0.005750,0.325915,0.631388,0.320516,0.399818,0.897504,...,0.741222,1,0,0,0,0.002032,0.008334,0.012686,0.424749,brain
6332,1.0,0.310709,0.942876,0.239702,0.108149,0.797971,0.846226,0.860231,0.267768,0.618295,...,0.390748,0,0,0,0,0.001661,0.003489,0.013848,0.464231,brain
8895,1.0,0.426903,0.653419,0.891579,0.071327,0.821819,0.487696,0.951512,0.909627,0.499259,...,0.755605,0,0,0,0,0.013962,0.000439,0.050616,0.467053,brain
5351,1.0,0.694429,0.317972,0.739563,0.503240,0.682732,0.862864,0.071619,0.569904,0.072986,...,0.829675,0,0,0,0,0.115543,0.000010,0.007895,0.345829,brain
4314,1.0,0.083574,0.708506,0.144303,0.614994,0.856661,0.673351,0.015961,0.647787,0.454005,...,0.816384,0,0,0,0,0.440379,0.007265,0.033471,0.418360,brain


In [16]:
import sklearn
from sklearn import metrics
for i in range(5):
    print(sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["SPEER"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["SPEER without transfer"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["tissue specific genome only"]),
          sklearn.metrics.roc_auc_score(test_list[i]["z_label"], test_list[i]["shared tissue genome only"]))

0.956462445614 0.909309527637 0.878462531284 0.886400248235
0.953230904317 0.861780868185 0.850184204187 0.877928784695
0.955591826384 0.838871416966 0.892336475331 0.885954844293
0.954174737822 0.888998406365 0.888931507087 0.876364309868
0.954830748182 0.830268423252 0.853996210307 0.87994036646


In [ ]:
sklearn.metrics.roc_auc_score(test_list[0]["z_labels"], test_list[0]["SPEER without transfer"])

In [17]:
bn = bnchmk.BenchmarkPosteriors(p.train_list, p.test_list, p.genomic_features)